# 第１ステージ　ステップ1-10

In [1]:
import numpy as np

## ステップ１

### 1.1変数とは
- 省略

### 1.2 Variableクラスの実装

In [2]:
class Variable:
    def __init__(self, data):
        self.data = data

In [3]:
data = np.array(1.0)
x = Variable(data)
x.data

array(1.)

In [4]:
x.data = np.array(2.)
x.data

array(2.)

### 1.3 【補足】Nunpyの多次元配列
- 省略

## ステップ２

### 2.2 Functionクラスの実装

In [5]:
class Function:
    def __call__(self, input):
        x = input.data # データを取り出す
        y = x ** 2 # 実際の計算
        output = Variable(y) # Variableとして返す
        return output

In [6]:
x = Variable(np.array(10))
f = Function()
y = f(x)

In [7]:
print(type(y))
print(y.data)

<class '__main__.Variable'>
100


### 2.3 Functionクラスを使う

1. Functionクラスは基底クラスとして、全ての関数に共通する機能を実装する
2. 具体的な関数は、Functionクラスを継承したクラスで実装する

以上の点を考慮し、Functionクラスを実装

In [8]:
class Function:
    def __call__(self, input):
        x = input.data
        y = self.forward(x) # 具体的な計算はforwardメソッドで行う。
        output = Variable(y)
        return output
    
    def forward(self, x):
        # 例外処理を行い、このクラスを継承すべきであるとアピール
        raise NotImplementedError()

- Functionクラスを継承して、Squareクラスを実装してみる

In [9]:
class Square(Function):
    def forward(self, x):
        return x ** 2

In [10]:
x = Variable(np.array(10))
f = Square()
y = f(x)
print(type(y))
print(y.data)

<class '__main__.Variable'>
100


## ステップ３　関数の連結

### 3.1 Exp関数の実装

- Functionクラスを継承して、$y=e^x$を実装

In [11]:
class Exp(Function):
    def forward(self, x):
        return np.exp(x)

### 3.2関数を連結する
- Functionクラスを継承したクラスを連結して、$y=(e^{x^2})^2$を実装
<br>
- 実装の流れ<br>
$x$ &rArr; Square() &rArr; $x^2$ &rArr; Exp() &rArr; $e^{x^2}$ &rArr; Square() &rArr; $(e^{x^2})^2$

In [12]:
A = Square()
B = Exp()
C = Square()

x = Variable(np.array(0.5))
a = A(x) # x^2
b = B(a) # e^(x^2)
y = C(b) # (e^(x^2))^2
print(y.data)

1.648721270700128


## ステップ５　バックプロパゲーションの理論
- チェインルール<br>
3.2の場合で$\frac{dy}{dx}$を計算してみる<br>
- 微分の公式通りであれば以下のようになる。
$$lim_{h \to 0} \frac{f(x+h)-f(x)}{h}$$
- 公式通りに微分することの問題点
1. 誤差が含まれやすい
2. 計算コストが大きすぎる。ニューラルネットの場合更新するパラメータが膨大

そこでチェインルールによるバックプロパゲーションの出番
<br>

3.2では順伝播は以下のようになっている<br>
$x$ &rArr; *A* &rArr; $x^2$ &rArr; *B* &rArr; $e^{x^2}$ &rArr; *C* &rArr; $(e^{x^2})^2$

ここで、
$$\frac{dy}{dx}=((\frac{dy}{dy}\frac{dy}{db})\frac{db}{da}\frac{da}{dx})$$
であることから、順伝播と逆伝播は以下のような関係といえる。<br>
$x$ &rArr; *A* &rArr; $a$ &rArr; *B* &rArr; $b$ &rArr; *C* &rArr; $y$<br>
<br>
$\frac{dy}{dx}=((\frac{dy}{dy}\frac{dy}{db})\frac{db}{da})\frac{da}{dx}$ &lArr; $A'(x)$ &lArr; $\frac{dy}{da}=(\frac{dy}{dy}\frac{dy}{db})\frac{db}{da}$ &lArr; $B'(a)$ &lArr; $\frac{dy}{db}=\frac{dy}{dy}\frac{dy}{db}$ &lArr; $C'(b)$ &lArr; $\frac{dy}{dy}=1$


## ステップ６　手作業によるバックプロパゲーション

### 6.1 Variableクラスの追加実装
- Variableクラスを通常の値(data)に加えて、それに対応する微分した値(grad)を持つよう拡張

In [13]:
class Variable:
    def __init__(self, data):
        self.data = data
        self.grad = None

### 6.2 Functionクラスの追加実装
- Functionクラスに２つの機能を追加
1. 微分の計算を行う逆伝播の機能(backwardメソッド)
2. forwardメソッドを呼ぶ際に、入力されたVariableインスタンスを保持する機能

In [14]:
class Function:
    def __call__(self, input):
        x = input.data
        y = self.forward(x) # 具体的な計算はforwardメソッドで行う。
        output = Variable(y)
        self.input = input # 入力された変数を覚える
        return output
    
    def forward(self, x):
        # 例外処理を行い、このクラスを継承すべきであるとアピール
        raise NotImplementedError()
    
    def backward(self, gy):
        raise NotImplementedError()

### 6.3 SquareとExpクラスの追加実装

In [15]:
class Square(Function):
    def forward(self, x):
        y = x ** 2
        return y
    
    def backward(self, gy):
        x = self.input.data
        gx = 2 * x * gy
        return gx

In [16]:
class Exp(Function):
    def forward(self, x):
        y = np.exp(x)
        return y
    
    def backward(self, gy):
        x = self.input.data
        gx = np.exp(x) * gy
        return gx

### 6.4 バックプロパゲーションの実装

In [17]:
# 順伝播の実装
A = Square()
B = Exp()
C = Square()

x = Variable(np.array(0.5))
a = A(x) # x^2
b = B(a) # e^(x^2)
y = C(b) # (e^(x^2))^2

In [18]:
# 逆伝播の実装
y.grad = np.array(1.)
b.grad = C.backward(y.grad)
a.grad = B.backward(b.grad)
x.grad = A.backward(a.grad)
print(x.grad)

3.297442541400256


In [19]:
# 定義通りの微分
def f(x):
    A = Square()
    B = Exp()
    C = Square()
    return C(B(A(x)))

def numerical_diff(f, x, eps=1e-4):
    x1 = Variable(x.data + eps)
    y1 = f(x1)
    y0 = f(x)
    return (y1.data - y0.data) / eps

h = np.array(1e-5)
x = Variable(np.array(0.5))
x_dif = numerical_diff(f, x)
print(x_dif)

3.2981021178524195


## ステップ７　バックプロパゲーションの自動化

### 7.1 逆伝播の自動化のために
#### 変数と関数の関係性
関数から見ると、変数は「入力」と「出力」として存在する。<br>
逆に、変数から見ると、関数は変数にとって<strong>生みの親(creator)</strong>である。<br>

そこで、通常の計算が行われるタイミングで、変数と関数のつながりを作る。<br>
VariableクラスとFunctionクラスを改良する。


In [20]:
class Variable:
    def __init__(self, data):
        self.data = data
        self.grad = None
        self.creator = None
    
    def set_creator(self, func):
        self.creator = func

In [24]:
class Function:
    def __call__(self, input):
        x = input.data
        y = self.forward(x)
        output = Variable(y)
        output.set_creator(self) # 出力変数に生みの親である関数を覚えさせる
        self.input = input
        self.output = output # 出力も覚える
        return output
    
    def forward(self, x):
    # 例外処理を行い、このクラスを継承すべきであるとアピール
        raise NotImplementedError()
    
    def backward(self, gy):
        raise NotImplementedError()

In [25]:
# VariableとFunctionで「つながり」が形成できていることを確認してみる
class Exp(Function):
    def forward(self, x):
        y = np.exp(x)
        return y
    
    def backward(self, gy):
        x = self.input.data
        gx = np.exp(x) * gy
        return gx
    
class Square(Function):
    def forward(self, x):
        y = x ** 2
        return y
    
    def backward(self, gy):
        x = self.input.data
        gx = 2 * x * gy
        return gx

In [26]:
A = Square()
B = Exp()
C = Square()

x = Variable(np.array(0.5))
a = A(x)
b = B(a)
y = C(b)

In [35]:
print(y.creator == C)
print(y.creator.input == b)
print(y.creator.input.creator == B)
print(y.creator.input.creator.input == a)
print(y.creator.input.creator.input.creator == A)
print(y.creator.input.creator.input.creator.input == x)

True
True
True
True
True
True


### 7.2 逆伝播を試す

In [36]:
y.grad = np.array(1.)

C = y.creator # 1. 関数を取得
b = C.input # 2. 関数の入力を取得
b.grad = C.backward(y.grad) # 3. 関数のbackwardメソッドを呼ぶ

In [37]:
B = b.creator
a = B.input
a.grad = B.backward(b.grad)

In [38]:
A = a.creator
x = A.input
x.grad = A.backward(a.grad)
print(x.grad)

3.297442541400256


### 7.3 backwardメソッドの追加
上記の逆伝播のコードには、同じ処理フローが繰り返し現れる。<br>
そこで、繰り返しの処理を自動化できるよう、Variableクラスを改良。

In [39]:
class Variable:
    def __init__(self, data):
        self.data = data
        self.grad = None
        self.creator = None
    
    def set_creator(self, func):
        self.creator = func
    
    def backward(self):
        f = self.creator # 1. 関数を取得
        if f is not None: # creatorがNoneの場合、逆伝播はストップ
            x = f.input # 2. 関数の入力を取得
            x.grad = f.backward(self.grad) # 3. 関数のbackwardメソッドを呼ぶ
            x.backward() # 4. 自分より１つ前の変数のbackwardを呼ぶ(再帰)

In [40]:
A = Square()
B = Exp()
C = Square()

x = Variable(np.array(0.5))
a = A(x)
b = B(a)
y = C(b)

# backward
y.grad = np.array(1.0)
y.backward()
print(x.grad)

3.297442541400256


## ステップ８　再帰からループへ
### 8.2 ループを使った実装
先ほどの「再帰を使った実装」を「ループを使った実装」に書き換える

In [41]:
class Variable:
    def __init__(self, data):
        self.data = data
        self.grad = None
        self.creator = None

    def set_creator(self, func):
        self.creator = func

    def backward(self):
        funcs = [self.creator]
        while funcs:
            f = funcs.pop()  # 1. 関数を取得
            x, y = f.input, f.output  # 2. 関数の入出力を取得
            x.grad = f.backward(y.grad)  # 3. backwardメソッドを呼ぶ

            if x.creator is not None:
                funcs.append(x.creator) # 4. １つ前の関数をリストに追加

- memo<br>
リストのpopメソッドは、リストの末尾が削除され、その要素が取得される。<br>
これを使ってループが組める。以下は簡単な例

In [47]:
x = [1] # self.creatorの結果が１とする
f_x = x.pop() # リストの１をf_xへ代入。このときxは空のリストになる。
print(f'f_x: {f_x}, x: {x}')
x.append(2) # 1つ前の関数(この場合２)をリストへ追加、xは空のリストではなくなるので、whileループに戻る

f_x: 1, x: []


### 8.3 動作確認

In [48]:
A = Square()
B = Exp()
C = Square()

x = Variable(np.array(0.5))
a = A(x)
b = B(a)
y = C(b)

# backward
y.grad = np.array(1.0)
y.backward()
print(x.grad)

3.297442541400256


## ステップ９　関数をより便利に
### 9.1 Pythonの関数として利用
現在、Squareクラスを使って計算するには、次のようにする必要がある。
```
x = Variable(np.array(.5))
f = Square()
y = f(x)
```
Squareクラスのインスタンスを生成、その後インスタンスを呼び出すという２段階のステップ<br>
→これは面倒なので、改良する。

In [49]:
def square(x):
    return Square()(x)


def exp(x):
    return Exp()(x)

In [51]:
x = Variable(np.array(.5))
y = square(exp(square(x))) # 連続して適用

y.grad = np.array(1.0)
y.backward()
x.grad

3.297442541400256

### 9.2 backwardメソッドの簡略化
先ほどのコードにある`y.grad = np.array(1.0)`は、毎回逆伝播する際書いている。<br>
->省略したいのでVariableクラスを改良

In [54]:
class Variable:
    def __init__(self, data):
        self.data = data
        self.grad = None
        self.creator = None

    def set_creator(self, func):
        self.creator = func

    def backward(self):
        if self.grad is None:
            # gradがNoneの場合、自動で微分を生成する
            # self.dataと同じ形状、データ型かつその要素が１のインスタンスを作成
            self.grad = np.ones_like(self.data)

        funcs = [self.creator]
        while funcs:
            f = funcs.pop()
            x, y = f.input, f.output
            x.grad = f.backward(y.grad)

            if x.creator is not None:
                funcs.append(x.creator)

In [55]:
x = Variable(np.array(0.5))
y = square(exp(square(x)))
y.backward()
print(x.grad)

3.297442541400256


### 9.3 ndarrayだけを扱う
Variableにndarrayインスタンス以外のデータを入れた場合、即座にエラーを出すようにする。

In [60]:
class Variable:
    def __init__(self, data):
        if data is not None:
            if not isinstance(data, np.ndarray):
                raise TypeError(f'{type(data)} is not supported')

        self.data = data
        self.grad = None
        self.creator = None

    def set_creator(self, func):
        self.creator = func

    def backward(self):
        if self.grad is None:
            self.grad = np.ones_like(self.data)

        funcs = [self.creator]
        while funcs:
            f = funcs.pop()
            x, y = f.input, f.output
            x.grad = f.backward(y.grad)

            if x.creator is not None:
                funcs.append(x.creator)

In [61]:
x = Variable(np.array(1.0))  # OK
x = Variable(None)  # OK
x = Variable(1.0)  # NG

TypeError: <class 'float'> is not supported

- memo<br>
このままでは、numpyの仕様によりエラーを吐く可能性あり。<br>
以下がその例。

In [62]:
# 期待通りの結果(yのデータ型もndarrayになっている)
x = np.array([1.])
y = x ** 2
print(type(x), x.ndim)
print(type(y))

<class 'numpy.ndarray'> 1
<class 'numpy.ndarray'>


In [63]:
#想定外の結果(yのデータ型がndarrayになっていない)
x = np.array(1.)
y = x ** 2
print(type(x), x.ndim)
print(type(y))

<class 'numpy.ndarray'> 0
<class 'numpy.float64'>


- これを解決するために便利関数を作る。<br>
np.isscalar関数というnp.float64などのスカラ系の型を判定できるメソッドを使う。

In [69]:
# np.isscalarでxがndarrayインスタンスか判定できる。
print(np.isscalar(np.float64(1.)))
print(np.isscalar(2.0))
print(np.isscalar(np.array(1.0)))
print(np.isscalar(np.array([1., 2., 3])))

True
True
False
False


In [70]:
def as_array(x):
    if np.isscalar(x):
        return np.array(x)
    return x

In [71]:
class Function:
    def __call__(self, input):
        x = input.data
        y = self.forward(x)
        output = Variable(as_array(y))
        output.set_creator(self)
        self.input = input
        self.output = output
        return output

    def forward(self, x):
        raise NotImplementedError()

    def backward(self, gy):
        raise NotImplementedError()

In [72]:
class Square(Function):
    def forward(self, x):
        y = x ** 2
        return y

    def backward(self, gy):
        x = self.input.data
        gx = 2 * x * gy
        return gx


class Exp(Function):
    def forward(self, x):
        y = np.exp(x)
        return y

    def backward(self, gy):
        x = self.input.data
        gx = np.exp(x) * gy
        return gx


def square(x):
    return Square()(x)


def exp(x):
    return Exp()(x)

In [73]:
x = Variable(np.array(0.5))
y = square(exp(square(x)))
y.backward()
print(x.grad)

3.297442541400256


## ステップ１０　テストを行う